# Natural Language Processing : Classic to Deep Methods for Sentiment Analysis

## Resources

Bag-Of-Word and TF-IDF:

https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/

Recurrent Neural Networks (RNNs):

https://medium.com/towards-data-science/illustrated-guide-to-recurrent-neural-networks-79e5eb8049c9

Long Short Term Memory networks (LSTMs):

https://medium.com/towards-data-science/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

http://colah.github.io/posts/2015-08-Understanding-LSTMs/

Word embeddings:

http://jalammar.github.io/illustrated-word2vec/

In [223]:
import os
import numpy as np
import pandas as pd
import json

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [59]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/julien.dubois@Digital-
[nltk_data]     Grenoble.local/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Today we are going to tackle the sentiment analysis problem, a *text classification* problem. The idea is pretty simple : we want to automatically predict whether a text expresses positive or negative sentiments. To do so we will use the IMDB dataset, that contains 50000 movie reviews from the www.imdb.com website, and their corresponding sentiment : positive or negative. It is thus a binary classification problem, where we want to predict a binary target $y \in \{0,1\}$. We will go through different ways of encoding a text in a vectorial form $x \in \mathbb{R}^d$, as well as different classification models, from classic ways to modern deep learning models.

## Load the dataset

Load the dataset and explore a bit the data :

In [110]:
#Load and print the dataset
imdb_dataset_original=pd.read_csv('../data/IMDB Dataset.csv')
imdb_dataset = imdb_dataset_original.copy()
imdb_dataset.head(10)

,review,sentiment
0,I saw this movie last night after waiting ages...,positive
1,"This was, so far, the worst movie I have seen ...",negative
2,"In a time of magic, barbarians and demons abou...",positive
3,I had high expectations of this movie (the tit...,negative
4,This is a film of immense appeal to a relative...,negative
5,An hilariously accurate caricature of trying t...,positive
6,I watch most movies that Nick Mancuso is in be...,positive
7,This is one of those films that's more interes...,negative
8,A wonderful and gritty war film that focuses o...,positive
9,"Man, some of you people have got to chill. Thi...",positive


In [36]:
#Print first review:
print(imdb_dataset["review"][0])

I saw this movie last night after waiting ages and ages for it to be released here in Canada (still only in limited release). It was worth the wait and then some. I am a very avid reader of Margaret Laurence and was excited to see that this novel was being turned into a film. I actually ended up liking the movie better than the novel. I liked that the character of Bram Shipley was a bit less harsh, and that there seemed to be more of a love story between Hagar and Bram, which made the scenes at the end of Bram's life that much more moving. The loss seemed stronger. Hagar was not any more likable on film than in the book, but Ellen Burstyn was a genius in this role. She WAS Hagar through and through. Christine Horne was brilliant and has many more great things ahead I am sure. Her scenes with Cole Hauser were electrifying. I could go on and on, overall a 9 * out of 10. Fantastic and can't wait for it to come out on DVD, a must own for my collection!


In [37]:
#Print the two classes size
imdb_dataset['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

## Text preprocessing

As you can see the text is quite messy, and before encoding our text into features, we are going to go through different preprocessing steps in order to clean it:
* Removing the HTML tags.
* Removing other special characters : this means all non alphanumeric characters, including punctuation.
* Lowercase the text.
* Tokenization : split a text as a list of words now called tokens.
* Stemming : removing all the suffixes from conjugation, plural, ... In order to bring a word back to its root form. For example.
* Removing stopwords : words like 'to', 'a', 'the', ... are called stopwords, we remove them as they are too frequent words and generally just add noise.

Fill the following functions to perform each of these steps. You are free to use the libraries of your choice to do so. Try to not reinvent the wheel!

In [38]:
def remove_html_tags(text):
    """
    Input: str : A string to clean from html tags
    Output: str : The same string with html tags removed
    """
    CLEANR = re.compile('<.*?>')
    cleantext = re.sub(CLEANR, ' ', text)
    return cleantext

In [111]:
imdb_dataset['review']=imdb_dataset['review'].apply(remove_html_tags)
imdb_dataset.head(10)

,review,sentiment
0,I saw this movie last night after waiting ages...,positive
1,"This was, so far, the worst movie I have seen ...",negative
2,"In a time of magic, barbarians and demons abou...",positive
3,I had high expectations of this movie (the tit...,negative
4,This is a film of immense appeal to a relative...,negative
5,An hilariously accurate caricature of trying t...,positive
6,I watch most movies that Nick Mancuso is in be...,positive
7,This is one of those films that's more interes...,negative
8,A wonderful and gritty war film that focuses o...,positive
9,"Man, some of you people have got to chill. Thi...",positive


In [112]:
def remove_special_characters(text):
    """
    Input: str : A string to clean from non alphanumeric characters
    Output: str : The same strings without non alphanumeric characters
    """
    cleantext = re.sub(r'[^a-zA-Z\d\s:]', ' ', text)
    return cleantext

In [113]:
imdb_dataset['review']=imdb_dataset['review'].apply(remove_special_characters)
imdb_dataset.head(10)

,review,sentiment
0,I saw this movie last night after waiting ages...,positive
1,This was so far the worst movie I have seen ...,negative
2,In a time of magic barbarians and demons abou...,positive
3,I had high expectations of this movie the tit...,negative
4,This is a film of immense appeal to a relative...,negative
5,An hilariously accurate caricature of trying t...,positive
6,I watch most movies that Nick Mancuso is in be...,positive
7,This is one of those films that s more interes...,negative
8,A wonderful and gritty war film that focuses o...,positive
9,Man some of you people have got to chill Thi...,positive


In [114]:
def lowercase_text(text):
    """
    Input: str : A string to lowercase
    Output: str : The same string lowercased
    """
    return text.lower()

In [115]:
imdb_dataset['review']=imdb_dataset['review'].apply(lowercase_text)
imdb_dataset.head(10)

,review,sentiment
0,i saw this movie last night after waiting ages...,positive
1,this was so far the worst movie i have seen ...,negative
2,in a time of magic barbarians and demons abou...,positive
3,i had high expectations of this movie the tit...,negative
4,this is a film of immense appeal to a relative...,negative
5,an hilariously accurate caricature of trying t...,positive
6,i watch most movies that nick mancuso is in be...,positive
7,this is one of those films that s more interes...,negative
8,a wonderful and gritty war film that focuses o...,positive
9,man some of you people have got to chill thi...,positive


In [116]:
def single_white_space(text):
    """
    Input: str : A string to clean from multiple white spaces in a row.
    Output: str : The same string with single white spaces.
    """
    cleantext = re.sub(r'\s+', ' ', text)
    return cleantext

In [117]:
imdb_dataset['review']=imdb_dataset['review'].apply(single_white_space)
imdb_dataset.head(10)

,review,sentiment
0,i saw this movie last night after waiting ages...,positive
1,this was so far the worst movie i have seen in...,negative
2,in a time of magic barbarians and demons aboun...,positive
3,i had high expectations of this movie the titl...,negative
4,this is a film of immense appeal to a relative...,negative
5,an hilariously accurate caricature of trying t...,positive
6,i watch most movies that nick mancuso is in be...,positive
7,this is one of those films that s more interes...,negative
8,a wonderful and gritty war film that focuses o...,positive
9,man some of you people have got to chill this ...,positive


In [118]:
def tokenize_words(text):
    """
    Input: str : A string to tokenize
    Output: list of str : A list of the tokens splitted from the input string
    """
    tokens = text.strip().split(" ")
    return tokens

In [119]:
imdb_dataset['review']=imdb_dataset['review'].apply(tokenize_words)
imdb_dataset.head(10)

,review,sentiment
0,"[i, saw, this, movie, last, night, after, wait...",positive
1,"[this, was, so, far, the, worst, movie, i, hav...",negative
2,"[in, a, time, of, magic, barbarians, and, demo...",positive
3,"[i, had, high, expectations, of, this, movie, ...",negative
4,"[this, is, a, film, of, immense, appeal, to, a...",negative
5,"[an, hilariously, accurate, caricature, of, tr...",positive
6,"[i, watch, most, movies, that, nick, mancuso, ...",positive
7,"[this, is, one, of, those, films, that, s, mor...",negative
8,"[a, wonderful, and, gritty, war, film, that, f...",positive
9,"[man, some, of, you, people, have, got, to, ch...",positive


In [120]:
def remove_stopwords(token_list):
    """
    Input: list of str : A list of tokens
    Output: list of str : The new list with removed stopwords tokens
    """
    stop_words = set(stopwords.words('english'))
    token_list = [token for token in token_list if token not in stop_words]
    return token_list

In [121]:
imdb_dataset['review']=imdb_dataset['review'].apply(remove_stopwords)
imdb_dataset.head(10)

,review,sentiment
0,"[saw, movie, last, night, waiting, ages, ages,...",positive
1,"[far, worst, movie, seen, entire, life, seen, ...",negative
2,"[time, magic, barbarians, demons, abound, diab...",positive
3,"[high, expectations, movie, title, translated,...",negative
4,"[film, immense, appeal, relatively, well, defi...",negative
5,"[hilariously, accurate, caricature, trying, se...",positive
6,"[watch, movies, nick, mancuso, frankly, love, ...",positive
7,"[one, films, interesting, watch, academic, per...",negative
8,"[wonderful, gritty, war, film, focuses, inner,...",positive
9,"[man, people, got, chill, movie, artistic, gen...",positive


In [71]:
def stem_words(token_list, stemmer):
    """
    Input: list of str : A list of tokens to stem
    Output: list of str : The list of stemmed tokens
    """
    stem_words = [stemmer.stem(token) for token in token_list]
    return stem_words

In [72]:
stemmer = PorterStemmer()
imdb_dataset['review']=imdb_dataset['review'].apply(lambda x: stem_words(x, stemmer))
imdb_dataset.head(10)

,review,sentiment
0,"[saw, movi, last, night, wait, age, age, relea...",positive
1,"[far, worst, movi, seen, entir, life, seen, re...",negative
2,"[time, magic, barbarian, demon, abound, diabol...",positive
3,"[high, expect, movi, titl, translat, get, rid,...",negative
4,"[film, immens, appeal, rel, well, defin, group...",negative
5,"[hilari, accur, caricatur, tri, sell, script, ...",positive
6,"[watch, movi, nick, mancuso, frankli, love, gu...",positive
7,"[one, film, interest, watch, academ, perspect,...",negative
8,"[wonder, gritti, war, film, focus, inner, torm...",positive
9,"[man, peopl, got, chill, movi, artist, geniu, ...",positive


Let's join all that together and apply it to our dataset. The following function simply chains all the preprocessing steps you just implemented. 

It adds the `list_output` flag, if False it will reconcatenate all the preprocessed tokens into a single string (with spaces between tokens), if True it will keep each sentence as a list of tokens. Depending on the libraries you will use for the next steps, it can be useful to have one or the other representation.

In [124]:
def normalize_text_dataset(dataset, text_col_name = 'review', html_tags = True,
                           special_chars = True, lowercase = True , stemming = True , 
                           stopwords = True, white_space = True, list_output = False ):
    """
    Apply the choosen preprocessing steps to a corpus of texts and return the 
    preprocessed corpus. The list_output flag allows to return either a list
    of token, or a rejoined string with spaces between the preprocessed tokens.
    """
    def rejoin_text(token_list):
        return ' '.join(token_list)
    
    
    output = dataset.copy()
    
    if html_tags : 
        output[text_col_name] = output[text_col_name].apply(remove_html_tags)
        
    if special_chars :
        output[text_col_name] = output[text_col_name].apply(remove_special_characters)
        
    if lowercase :
        output[text_col_name] = output[text_col_name].apply(lowercase_text)
    
    if white_space:
        output[text_col_name] = output[text_col_name].apply(single_white_space)
        
    #Tokenization for next steps:
    output[text_col_name] = output[text_col_name].apply(tokenize_words)
    
    if stopwords :
        output[text_col_name] = output[text_col_name].apply(remove_stopwords)
        
    if stemming :
        stemmer = PorterStemmer()
        output[text_col_name] = output[text_col_name].apply(lambda x: stem_words(x, stemmer))
        
    if not list_output :
        output[text_col_name] = output[text_col_name].apply(rejoin_text)
        
    return output
        

In [172]:
imdb_clean_dataset = normalize_text_dataset(imdb_dataset_original, html_tags = True,
                           special_chars = True, lowercase = True , stemming = True , 
                           stopwords = True, list_output = False )

## Text classification with Bag-Of-Words

Now we have cleaned the reviews of our dataset, how do we represent them as vectors in order to classify it ? 
One classic way to achieve that is the Bag-Of-Words (BOW) approach. To encode a text in a bag of word, we first need to know all the different words $w$ that appear in all our reviews, called the vocabulary : $w \in \mathcal{V}$. For each word $w$ we attribute an index $idx(w) = i$ with $i \in \{0, |\mathcal{V}|-1\}$, and represent a review $r$ as a vector of the size of the vocabulary $x_r \in \mathbb{R}^{|\mathcal{V}|}$. To encode a review we are simply going to count how many time each word appears and assign it at its corresponding index in the bag-of-words vector : $x_{r,i} = count(w,r)$, where i = idx(w). 

This means that we completely disregard the words order, and simply take into account the number of times each word appears in each review to represent them. There are many variations of this concept, TF-IDF (term frequency-inverse document frequency) for example, gives more weight to uncommon words. Read more about BOW and TF-IDF there:

https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/

Let's start with bag-of-words. In general we don't consider the whole vocabulary but only some of the most frequent words in order to reduce the dimensionality and avoid noise from rare words. Here we will only consider the 10000 most frequent words of the training set, meaning the words that are only in the test set will be ignored. Thus we have : $x_r \in \mathbb{R^{10000}}$.

Encode all the reviews as bag-of-words, and train and evaluate a logistic regression model on the following train test splits. As we have seen previously, if we wanted to investigate this model we should also grid search for hyperparameters by doing a cross-validation with validation sets, etc. However this is not the goal today, so we'll simply go for a train/test split for this experiment. Concerning the evaluation metrics, in this case we care equally about correctly predicting the positives and the negatives, and we have a balanced dataset, thus we can simply use accuracy this time.

Once again, don't do everything from scratch and try to find libraries that propose implementations of these concepts !

In [ ]:
max_vocab_size = 10000 

#Train/test split:
lb=LabelBinarizer()
sentiment_labels=lb.fit_transform(imdb_clean_dataset['sentiment'])

train_reviews = imdb_clean_dataset.review[:25000]
test_reviews = imdb_clean_dataset.review[25000:]

train_sentiments = sentiment_labels[:25000]
test_sentiments = sentiment_labels[25000:]

In [210]:
# Vocabulary + encoding
vectorizer = CountVectorizer(
    input='content',
    max_features=max_vocab_size
    )

X_train = vectorizer.fit_transform(train_reviews.values)
X_test = vectorizer.transform(test_reviews.values)

In [211]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, train_sentiments.ravel())
y_pred = model.predict(X_test)

In [212]:
model_accuracy = dict()
model_accuracy["bow"] = accuracy_score(test_sentiments, y_pred)
print(f"The bag-of-words method yielded an accuracy of {100*model_accuracy["bow"]:.1f}%.")

The bag-of-words method yielded an accuracy of 84.8%.


You should get about 88% accuracy, pretty good for such a simple model. Now let's do the same with a tf-idf encoding:

In [ ]:
TFIDF_vectorizer = TfidfVectorizer(
    input='content',
    max_features=max_vocab_size
    )

X_train = TFIDF_vectorizer.fit_transform(train_reviews.values)
X_test = TFIDF_vectorizer.transform(test_reviews.values)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, train_sentiments.ravel())
y_pred_tfidf = model.predict(X_test)

In [224]:
model_accuracy["tf-idf"] = accuracy_score(test_sentiments, y_pred_tfidf)
print(f"The TF-IDF method yielded an accuracy of {100*model_accuracy["tf-idf"]:.1f}%.")

The TF-IDF method yielded an accuracy of 87.9%.


And you should get about 90% accuracy this time. Other classic but more sophisticated features include N-grams, part-of-speech tagging and syntax trees, you can read more about these there:

https://www.analyticsvidhya.com/blog/2020/07/part-of-speechpos-tagging-dependency-parsing-and-constituency-parsing-in-nlp/

But we will stop there for the classic approaches and go to deep learning methods.

...unless you are ahead of time, in this case learn about Bag of N-grams by yourself, and try them out :

In [ ]:
#TOFILL (Optional)
model_accuracy
with open('../data/model_accuracy.json', 'w') as f:
    f.write(json.dumps(model_accuracy))